<a href="https://colab.research.google.com/github/av-gav/itmo-nlp/blob/main/Task2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Putting it all together (TensorFlow)

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] accelerate pycountry

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip drive/MyDrive/train/lid.zip

In [5]:
import pycountry
import datasets

wili = datasets.load_dataset('wili_2018')
# names for languages not in iso-639-3 from wikipedia
non_iso_languages = {'roa-tara': 'Tarantino', 'zh-yue': 'Cantonese', 'map-bms': 'Banyumasan',
                      'nds-nl': 'Dutch Low Saxon', 'be-tarask': 'Belarusian-Tarask'}

diff = {'Russia Buriat':'Buriat', 'Kirghiz':'Kyrgyz', 'Ossetian':'Ossetic'}
# create dictionary from data set labels to language names
lab2lang = {}
for i, lang in enumerate(wili['train'].features['label'].names):
  full_lang = pycountry.languages.get(alpha_3=lang)
  if full_lang:
    lab2lang[i] = diff.get(full_lang.name, full_lang.name)
  else:
    lab2lang[i] = non_iso_languages[lang]

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from datasets import ClassLabel
ds = datasets.load_from_disk('lid')

features = ds['train'].features.copy()
class_label = features['label']
langs = class_label.names
id2label = {idx: langs[idx] for idx in range(len(langs))}
label2id = {v: k for k,v in id2label.items()}
common = [label2id[x] for x in langs if x in lab2lang.values()]
features['label'] = ClassLabel(names=[lab2lang[k] for k in sorted(lab2lang)])
mapper = {label2id[v]:k for k,v in lab2lang.items() if v in label2id}
def map_lang(x):
  x['label'] = mapper[x['label']]
  return x
ds_part = ds.filter(lambda x: x['label'] in common).cast(features).map(map_lang)

In [8]:
n_labs = len(lab2lang)
id2lab = {i:f'LAB_{i}' for i in range(n_labs)}
lab2id = {v:k for k,v in id2lab.items()}

In [9]:
from transformers import CanineTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = CanineTokenizer.from_pretrained("google/canine-c", padding="longest")

model = AutoModelForSequenceClassification.from_pretrained("SebOchs/canine-c-lang-id",
                                                             num_labels=n_labs, id2label=id2lab, label2id=lab2id)

pipe = pipeline("text-classification",
                tokenizer = tokenizer,
                model = model,
                devoce = 0)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [53]:
import evaluate

metric = evaluate.load('accuracy')
evaluator = evaluate.evaluator("text-classification")

result = evaluator.compute(model_or_pipeline=pipe, 
                           data=ds_part['test'].select(range(1000)), metric=metric, label_mapping=lab2id)

print(result)


{'accuracy': 0.794, 'total_time_in_seconds': 271.81565637399945, 'samples_per_second': 3.6789639468893194, 'latency_in_seconds': 0.27181565637399946}


In [10]:
from transformers import DataCollatorWithPadding, TrainingArguments

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding='max_length')

tokenized_ds = ds.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("test-trainer",
                                  auto_find_batch_size =True,
                                  )

n_labs = len(langs)
model = AutoModelForSequenceClassification.from_pretrained("SebOchs/canine-c-lang-id",
                                                           ignore_mismatched_sizes=True,
                                                           num_labels=n_labs, 
                                                           id2label=id2label, label2id=label2id)

Map:   0%|          | 0/41594 [00:00<?, ? examples/s]

Map:   0%|          | 0/10399 [00:00<?, ? examples/s]

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at SebOchs/canine-c-lang-id and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([235, 768]) in the checkpoint and torch.Size([36, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([235]) in the checkpoint and torch.Size([36]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
